In [11]:
import yfinance as yf
import pandas as pd
from pathlib import Path
import math

In [12]:
result_folder = Path.cwd().parent / 'Results'

symbols = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker'].tolist()
symbols = pd.Series(symbols).unique()
backup = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker']
backup.to_csv(result_folder / 'known_tickers.csv', index=False)

In [13]:
df = pd.DataFrame(columns=[
    'Ticker', 
    'short_name', 
    'value_score',  
    'analyst_mean%', 
    'insider_buy%', 
    'sector', 
    'industry', 
    'P/E (TTM)', 
    'ROA%'
])

for symbol in symbols:
    ticker = yf.Ticker(symbol)
    try:
        analyst_mean = 0
        if 'mean' in ticker.analyst_price_targets and ticker.info['currentPrice']:
            analyst_mean = ticker.analyst_price_targets['mean'] / ticker.info['currentPrice'] - 1
        
        roa = ticker.info['returnOnAssets']*100
        pe = ticker.info['trailingPE']
        
        value_score = format((2.5 + math.log(roa) - math.log(pe)), '.2f')
        
        insider_buy = 0
        if 'Shares' in ticker.insider_purchases.columns:
            insider_buy = format(ticker.insider_purchases.loc[ticker.insider_purchases.index[4], 'Shares'], '.2f')
        
        df.loc[symbol] = [
            symbol, 
            ticker.info['shortName'], 
            value_score,
            format(analyst_mean * 100, '.2f'), 
            insider_buy,
            ticker.info['sector'], 
            ticker.info['industry'], 
            format(pe, '.2f') if isinstance(pe, (int, float)) else 'N/A', 
            format(roa, '.2f') if isinstance(roa, (int, float)) else 'N/A',
        ]
    except Exception as e:
        print(f"Error with {symbol}: {e}. Removed Ticker from list.")

df.to_csv(result_folder / 'simple_screener_results.csv', index=False)
display(df)

print(f'Got data for {len(df)} stocks')

,Ticker,short_name,value_score,analyst_mean%,insider_buy%,sector,industry,P/E (TTM),ROA%
MATAS.CO,MATAS.CO,Matas A/S,0.77,31.77,<NA>,Consumer Cyclical,Specialty Retail,21.65,3.84
TRIFOR.CO,TRIFOR.CO,Trifork Group AG,0.40,36.35,<NA>,Technology,Information Technology Services,12.92,1.59
QQ.L,QQ.L,QINETIQ GROUP PLC ORD 1P,1.41,1.68,<NA>,Industrials,Aerospace & Defense,20.57,6.89
RNMBY,RNMBY,Rheinmetall AG,-0.42,0.00,<NA>,Industrials,Aerospace & Defense,110.56,5.97
DRS,DRS,"Leonardo DRS, Inc.",0.39,22.73,0.00,Industrials,Aerospace & Defense,38.19,4.64
SAABF,SAABF,Saab AB,-0.15,0.00,<NA>,Industrials,Aerospace & Defense,52.43,3.70
BCKIY,BCKIY,Babcock International Group Plc,1.05,0.00,<NA>,Industrials,Engineering & Construction,21.70,5.08
BAESY,BAESY,BAE Systems PLC,0.69,-12.52,<NA>,Industrials,Aerospace & Defense,26.81,4.40
IVSO.ST,IVSO.ST,INVISIO AB,1.38,-17.66,<NA>,Technology,Communication Equipment,62.50,20.30
NSKFF,NSKFF,Kongsberg Gruppen ASA,0.27,0.00,<NA>,Industrials,Aerospace & Defense,62.19,6.66


Got data for 28 stocks


In [14]:
print(yf.Ticker('RNMBY').info.keys())

dict_keys(['address1', 'city', 'zip', 'country', 'phone', 'fax', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'fullTimeEmployees', 'companyOfficers', 'compensationAsOfEpochDate', 'executiveTeam', 'maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'dividendYield', 'exDividendDate', 'payoutRatio', 'fiveYearAvgDividendYield', 'beta', 'trailingPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 'bid', 'ask', 'bidSize', 'askSize', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage', 'twoHundredDayAverage', 'trailingAnnualDividendRate', 'trailingAnnualDividendYield', 'currency', 'tradeable', 'enterpriseValue', 'profitMargins', 'floatShares', 'sharesOutstanding', 'heldPercentInsiders', 'heldPer